<a href="https://colab.research.google.com/github/jingmingliu01/DeepLearning-DSCI-6011-01-25Fall/blob/main/deepleaning_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class NN(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear1 = nn.Linear(2,5,bias=True)
    self.linear2 = nn.Linear(5,3,bias=True)
    # ReLU already defined as function

  def forward(self,x):
    p=self.linear1.forward(x)
    h=F.relu(p)
    z=self.linear2.forward(h)
    return z

In [3]:
model = NN()

In [6]:
CELoss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.01)

In [13]:
class RandDataset(torch.utils.data.Dataset):
  def __init__(self,nSamples):
    super().__init__()
    self.X = torch.rand((nSamples,2))
    self.y = torch.randint(low=0,high=3,size=(nSamples,))

  def __len__(self):
    sz = self.X.shape[0]
    return sz

  def __getitem__(self,idx):
    sample = self.X[idx]
    label = self.y[idx]
    return sample,label

In [14]:
train_dataset = RandDataset(nSamples=100)
val_dataset = RandDataset(nSamples=20)

In [18]:
train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=8,shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset,batch_size=8,shuffle=False)

In [24]:
best_val_acc = 0

for epoch in range(10):
  print("epoch",epoch)
  model.train()
  for batch in train_dataloader:
    samples, labels = batch
    print("sample size",samples.shape)
    zhat = model.forward(samples)
    loss = CELoss(zhat,labels)
    print("loss",loss.item())
    model.zero_grad()
    loss.backward()
    optimizer.step()

  model.eval()
  with torch.no_grad():
    correct = 0
    for batch in val_dataloader:
      samples, labels = batch
      zhat = model.forward(samples)
      loss = CELoss(zhat,labels)
      print("test loss",loss.item())
      pred_labels = torch.argmax(zhat,dim=1)
      correct += (pred_labels==labels).sum()
    val_acc = correct/len(val_dataset)
    print("val acc",val_acc)
    if val_acc > best_val_acc:
      best_val_acc = val_acc
      torch.save(model.state_dict(),"best_model.pt")

epoch 0
sample size torch.Size([8, 2])
loss 1.1084927320480347
sample size torch.Size([8, 2])
loss 1.0904924869537354
sample size torch.Size([8, 2])
loss 1.1054518222808838
sample size torch.Size([8, 2])
loss 1.0589953660964966
sample size torch.Size([8, 2])
loss 1.0734666585922241
sample size torch.Size([8, 2])
loss 1.0557841062545776
sample size torch.Size([8, 2])
loss 1.080081820487976
sample size torch.Size([8, 2])
loss 1.1085764169692993
sample size torch.Size([8, 2])
loss 1.0782756805419922
sample size torch.Size([8, 2])
loss 1.0352636575698853
sample size torch.Size([8, 2])
loss 1.0946829319000244
sample size torch.Size([8, 2])
loss 1.0967724323272705
sample size torch.Size([4, 2])
loss 1.1027653217315674
test loss 1.1037178039550781
test loss 1.1149903535842896
test loss 1.1171094179153442
val acc tensor(0.2500)
epoch 1
sample size torch.Size([8, 2])
loss 1.0599502325057983
sample size torch.Size([8, 2])
loss 1.1195580959320068
sample size torch.Size([8, 2])
loss 1.099370956420

In [28]:
model2=NN()
model2.load_state_dict(torch.load("best_model.pt",weights_only=True))

<All keys matched successfully>